In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Conv2D, Input, MaxPooling2D, BatchNormalization, Add, Activation, SeparableConv2D, AveragePooling2D, Flatten
import matplotlib.pyplot as plt
from tensorflow import keras

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[]


In [3]:
data_dir="C:\\Users\\user\\PycharmProjects\\diplomaOflineModel\\data\\train"

In [ ]:

from os import listdir
from os.path import isfile, join
#import cv2
import os
dir_num=0
images_num=[]

dirnames=[data_dir+'\\'+ f for f in listdir(data_dir)]
for dirname in dirnames:
    dir_num+=1
    images_num1=0
    filenames=[dirname+'\\'+ f for f in listdir(dirname) if isfile(join(dirname,f))]
    for filename in filenames:
        try:
            images_num1+=1
            img = tf.io.read_file(filename)
            img = tf.io.decode_image(img, channels=3)
            
        except Exception:
            os.remove(filename)
            print("deleted err",filename)
    images_num.append(images_num1)
print(dir_num)
print(images_num)

In [ ]:
import statistics
import random
med=statistics.median(images_num)
for num in images_num:
    if num>med:
        dir=dirnames[images_num.index(num)]
        filenames=[dir+'\\'+ f for f in listdir(dir) if isfile(join(dir,f))]
        for i in range(num-round(med)):
            filename=random.choice(filenames)
            os.remove(filename)
            filenames.remove(filename)
            print("deleted",filename)

In [4]:
img_height, img_width=224, 224
batch_size=200
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
train_ds=train_ds.cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), tf.one_hot(y, 118)))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), tf.one_hot(y, 118)))
train_ds=train_ds.cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds=val_ds.cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

Found 46445 files belonging to 118 classes.
Using 37156 files for training.
Found 46445 files belonging to 118 classes.
Using 9289 files for validation.


In [6]:
def imshow():
    n=3
    plt.figure(figsize=(img_height,img_width))
    for images, labels in train_ds.take(1):
        for i in range(n):
            img = images[i]
            ax = plt.subplot(3, n, i+1+n)
            plt.imshow(img)
            plt.axis('off')
            plt.title(str(labels.numpy()[i]))
            ax.get_xaxis().set_visible(False)
    plt.show()
imshow()

In [5]:
def addAttoNetModule(deep1, deep2, deep3, deep4, layers, isS2=False):
    newLayers=BatchNormalization()(layers)
    newLayers=Activation('relu')(newLayers)
    newLayers=Conv2D(deep1, (1, 1), padding='same')(newLayers)
    newLayers=BatchNormalization()(newLayers)
    newLayers=Activation('relu')(newLayers)
    if isS2:
        newLayers=SeparableConv2D(deep2, (3, 3), padding='same', depth_multiplier=4)(newLayers)
    else:
        newLayers=SeparableConv2D(deep2, (3, 3), padding='same', depth_multiplier=4)(newLayers)
    newLayers=Conv2D(deep3, (1, 1), padding='same')(newLayers)
    newLayers=BatchNormalization()(newLayers)
    newLayers=Activation('relu')(newLayers)
    newLayers=Conv2D(deep4, (1, 1), padding='same')(newLayers)
    return newLayers

In [6]:
inputs=Input(shape=(img_height, img_width, 3))
x=Conv2D(8, (7, 7), activation='relu', strides=2, padding='same')(inputs)
x=MaxPooling2D((3, 3), strides=2, padding='same')(x)
#Module 1
xx=addAttoNetModule(8,32,16,176,x)
x=Conv2D(176, (1, 1), activation='relu', padding='same')(xx)
#Module 2
xx=addAttoNetModule(16,64,16,176,x)
x=Add()([xx, x])
#Module 3
xx=addAttoNetModule(16,64,16,176,x)
x=Add()([xx, x])
#Module 4
xx=addAttoNetModule(40,160,40,400,x,True)
x=Conv2D(400, (1, 1), activation='relu', padding='same', strides=2)(xx)
#Module 5
xx=addAttoNetModule(32, 128, 24, 400,x)
x=Add()([xx, x])
#Module 6
xx=addAttoNetModule(40, 160, 32, 400,x)
x=Add()([xx, x])
#Module 7
xx=addAttoNetModule(40, 160, 32, 400,x)
x=Add()([xx, x])
#Module 8
xx=addAttoNetModule(80, 320, 64, 808,x,True)
x=Conv2D(808, (1, 1), activation='relu', padding='same', strides=2)(xx)
#Module 9
xx=addAttoNetModule(72, 288, 72, 808,x)
x=Add()([xx, x])
#Module 10
xx=addAttoNetModule(80, 320, 72, 808,x)
x=Add()([xx, x])
#Module 11
xx=addAttoNetModule(72, 288, 72, 808,x)
x=Add()([xx, x])
#Module 12
xx=addAttoNetModule(72, 288, 64, 808,x)
x=Add()([xx, x])
#Module 13
xx=addAttoNetModule(56, 224, 56, 808,x)
x=Add()([xx, x])
#Module 14
xx=addAttoNetModule(160, 640, 128, 952,x,True)
x=Conv2D(952, (1, 1), activation='relu', padding='same', strides=2)(xx)
#Module 15
xx=addAttoNetModule(120, 480, 88, 952,x)
x=Add()([xx, x])
#Module 16
xx=addAttoNetModule(112, 448, 88, 952,x)
x=Add()([xx, x])

x=AveragePooling2D(7)(x)
x=Flatten()(x)
x=Dense(952, activation='relu')(x)
x=Dense(118, activation='sigmoid')(x)
outputs= x
cnn = keras.Model(inputs, outputs)

In [7]:
cnn.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │      1,184 │ input_layer[0][0] │
│                     │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 56, 56, 8) │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 56, 56, 8) │         32 │ max_pooling2d[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 56, 56, 8) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 56, 56, 8) │         72 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56, 8) │         32 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 56, 56, 8) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 56, 56,    │      1,344 │ activation_1[0][… │
│ (SeparableConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │        528 │ separable_conv2d… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │         64 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 56, 56,    │      2,992 │ activation_2[0][… │
│                     │ 176)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │     31,152 │ conv2d_3[0][0]    │
│                     │ 176)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        704 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 176)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 176)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 56, 56,    │      2,832 │ activation_3[0][… │
│                     │ 16)               │            │                 

 Total params: 6,013,966 (22.94 MB)

 Trainable params: 5,992,398 (22.86 MB)

 Non-trainable params: 21,568 (84.25 KB)

In [8]:
class Model(keras.Model):
    def __init__(self, nn):
        super(Model, self).__init__()
        self.nn=nn
        
    def get_loss(self, y, pred):
        loss=tf.keras.losses.categorical_crossentropy(y, pred)
        return loss
    
    @tf.function
    def training_step(self, x, y):
        with tf.GradientTape() as tape:
            preds = self.nn(x)
            loss=self.get_loss(y, preds)
        gradients = tape.gradient(loss, self.nn.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.nn.trainable_variables))
        return tf.reduce_mean(loss)

In [9]:
model= Model(cnn)
model.compile(optimizer= tf.keras.optimizers.Adam(1e-5))

In [ ]:
history = model.fit(train_ds, epochs=10,
                    validation_data=train_ds,
                    validation_steps=30)


In [ ]:
for x, y in train_ds.take(1):
    print(model.training_step(x, y))

In [ ]:
from IPython.display import clear_output
hist=np.array(np.empty([0]))
epochs=20
for i in range(1, epochs+1):
    loss=0
    print("Epoch {}/{}".format(i, epochs))
    for x, y in train_ds:
        print(model.training_step(x, y))
    clear_output(wait=True)
    hist =np.append(hist, loss)
    plt.plot(np.arange(0, 100), hist)
    plt.show()

Epoch 1/20
tf.Tensor(4.7706847, shape=(), dtype=float32)
tf.Tensor(4.770675, shape=(), dtype=float32)
tf.Tensor(4.770648, shape=(), dtype=float32)
tf.Tensor(4.7706456, shape=(), dtype=float32)
